In [1]:
%matplotlib inline

# Objective

The goal of this section of the project is harmonize and integrate traffic and weather data. 

The resulting integrated dataset will include the following fields:
* Camera ID
* Speed
* Weather status
* Timestamp
* Time of day (Hour)
* Is Daytime


# Data Integration

Now it's time to integrate our datasources. The goal is to combine:

* Camera Image features: This is the feature dataset we obtained in the pervious section
* Weather: For the time being, we're interested in datapoints that affect ligthing conditions such as: sunrise, sunset, rainfall and cloud cover
* Traffic speed data: We will get speed and location name data

We will export the combined dataset as CSV for further analysis. As a summary, the following data map illustrates the position of the cameras as well as certain key landmarks that are referenced in the datasources.

![](https://raw.githubusercontent.com/rmalarc/is602/master/final_project/data_map.jpg)

## The Code

In [2]:
#from __future__ import print_function

#import math
#import numpy as np
#import matplotlib.pyplot as plt
import os
import pandas as pd

### Traffic Data

Let's look at a sample of the previously downloaded traffic data.

In [3]:
traffic_data = pd.DataFrame().from_csv("camera_data/1447258654_speed_data.txt",sep="\t", index_col=False)
pd.options.display.max_colwidth =1000
traffic_data[['Id','TravelTime','Speed','linkName']].loc[traffic_data['linkName'].str.contains("VWE")]

,Id,TravelTime,Speed,linkName
32,171,255,47.85,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave
119,422,308,29.20,VWE N MP4.63 (Exit 6 - Jamaica Ave) - MP6.39 (Exit 11 Jewel Ave)
120,423,98,55.30,VWE N MP6.39 (Exit 11 Jewel Ave) - MP8.65 (Exit 13 Northern Blvd)
121,424,182,29.83,VWE N MP8.64 (Exit 13 Northern Blvd) - Whitestone Expwy Exit 14 (Linden Pl)
122,425,242,44.74,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkwy E 182nd St
123,426,121,57.17,VWE S MP4.63 (Exit 6 Jamaica Ave) - MP2.66 (Exit 2 Roackaway Blvd)
124,427,857,9.94,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exit 6 Jamaica Ave)
125,428,101,52.82,VWE S MP8.65 (Exit 13 Northern Blvd) - MP6.39 (Exit 11 Jewel Ave)
146,453,93,55.30,Whitestone Expwy S Exit 14 (Linden Pl) - VWE S MP8.65 (Exit 13 Northern Blvd)


As we can see above, there is traffic data available for multiple locations along the Van Wyck Expressway. Let's create a mapping table to keep the camera_id  to location_id 


In [4]:
location_to_camera_mapping = pd.DataFrame({'location_id':['427','422','426','426','171']
                                               ,'camera_id':['594','593','592','589','590']})
location_to_camera_mapping

,camera_id,location_id
0,594,427
1,593,422
2,592,426
3,589,426
4,590,171


With this mapping table, let's load all traffic data for the locations of interest into a dataframe:

In [5]:
traffic_df = pd.DataFrame() 
base_dir = "./camera_data/"


for filename in os.listdir(base_dir):
    if '_speed_data.txt' in filename:
        timestamp = filename[:10]

        tmp = pd.DataFrame().from_csv(base_dir+filename,sep="\t", index_col=False)

        # clean up the column names and data types
        tmp = tmp[['Id','TravelTime','Speed','linkName']]
        tmp.columns = ['location_id','travel_time','speed','location_name']
        tmp['location_id']=tmp[['location_id']].astype(str)
        
        # join against location_to_Camera_mapping table
        tmp = pd.merge(tmp, location_to_camera_mapping, how='inner', on='location_id')

        # set some extra attributes
        tmp['speed_data_filename']=filename
        tmp['timestamp']=timestamp

        traffic_df = traffic_df.append(tmp,ignore_index =True)

traffic_df.head()


,location_id,travel_time,speed,location_name,camera_id,speed_data_filename,timestamp
0,171,255,47.85,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,590,1447258654_speed_data.txt,1447258654
1,422,308,29.20,VWE N MP4.63 (Exit 6 - Jamaica Ave) - MP6.39 (Exit 11 Jewel Ave),593,1447258654_speed_data.txt,1447258654
2,426,121,57.17,VWE S MP4.63 (Exit 6 Jamaica Ave) - MP2.66 (Exit 2 Roackaway Blvd),592,1447258654_speed_data.txt,1447258654
3,426,121,57.17,VWE S MP4.63 (Exit 6 Jamaica Ave) - MP2.66 (Exit 2 Roackaway Blvd),589,1447258654_speed_data.txt,1447258654
4,427,857,9.94,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exit 6 Jamaica Ave),594,1447258654_speed_data.txt,1447258654


## Weather Data

Let's take a look at the weather data that we previously downloaded

In [6]:
import json

json_data=open("camera_data/1447258654_weather_data.json").read()

weather_data = json.loads(json_data)
weather_data

{u'base': u'cmc stations',
 u'clouds': {u'all': 92},
 u'cod': 200,
 u'coord': {u'lat': 40.7, u'lon': -73.8},
 u'dt': 1447256262,
 u'id': 5133268,
 u'main': {u'grnd_level': 1020.83,
  u'humidity': 100,
  u'pressure': 1020.83,
  u'sea_level': 1024.4,
  u'temp': 283.105,
  u'temp_max': 283.105,
  u'temp_min': 283.105},
 u'name': u'Queens County',
 u'rain': {u'3h': 0.3075},
 u'sys': {u'country': u'US',
  u'message': 0.0034,
  u'sunrise': 1447241862,
  u'sunset': 1447278021},
 u'weather': [{u'description': u'light rain',
   u'icon': u'10d',
   u'id': 500,
   u'main': u'Rain'}],
 u'wind': {u'deg': 355.505, u'speed': 4.92}}

From the above json, we're interested in any attributes that affect the visual appereance:
* weather summary
* sunrise
* sunset

In addition, we will define the following attributes:
* is_day: Flag that indicates if it's broad daylight based on the sunrise, sunset and current time.
* hour: Hour of day

In [7]:
print (weather_data['weather'][0]['main']
      ,weather_data['sys']['sunrise']
      ,weather_data['sys']['sunset'])

(u'Rain', 1447241862, 1447278021)


Let's now load all the weather data into a dataframe

In [8]:
import datetime

weather_data_df = pd.DataFrame() 
base_dir = "./camera_data/"

weather = []

for filename in os.listdir(base_dir):
    if '_weather_data.json' in filename:
        timestamp = filename[:10]

        tmp = pd.DataFrame()

        json_data=open(base_dir+filename).read()
        weather_data = json.loads(json_data)

        # Define derived attrobutes
        is_daytime =  weather_data['sys']['sunrise'] < int(timestamp) <weather_data['sys']['sunset']
        timestamp_dt = datetime.datetime.fromtimestamp(int(timestamp))
        hour = timestamp_dt.strftime('%H')
        
        weather.append({
                        "weather_description" : weather_data['weather'][0]['main'],
                        "is_daytime" : is_daytime,
                        "hour" : hour,
                        "weather_data_filename":filename,
                        "timestamp":timestamp
         })

weather_df = pd.DataFrame(weather)

weather_df.head()


,hour,is_daytime,timestamp,weather_data_filename,weather_description
0,11,True,1447258654,1447258654_weather_data.json,Rain
1,11,True,1447260454,1447260454_weather_data.json,Rain
2,12,True,1447262255,1447262255_weather_data.json,Clouds
3,12,True,1447264056,1447264056_weather_data.json,Clouds
4,13,True,1447265857,1447265857_weather_data.json,Clouds


# Merging Traffic and Weather Data

In [9]:
traffic_and_weather_df = pd.merge(traffic_df, weather_df, how='inner', on='timestamp')


In [10]:
traffic_and_weather_df.head()

,location_id,travel_time,speed,location_name,camera_id,speed_data_filename,timestamp,hour,is_daytime,weather_data_filename,weather_description
0,171,255,47.85,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,590,1447258654_speed_data.txt,1447258654,11,True,1447258654_weather_data.json,Rain
1,422,308,29.20,VWE N MP4.63 (Exit 6 - Jamaica Ave) - MP6.39 (Exit 11 Jewel Ave),593,1447258654_speed_data.txt,1447258654,11,True,1447258654_weather_data.json,Rain
2,426,121,57.17,VWE S MP4.63 (Exit 6 Jamaica Ave) - MP2.66 (Exit 2 Roackaway Blvd),592,1447258654_speed_data.txt,1447258654,11,True,1447258654_weather_data.json,Rain
3,426,121,57.17,VWE S MP4.63 (Exit 6 Jamaica Ave) - MP2.66 (Exit 2 Roackaway Blvd),589,1447258654_speed_data.txt,1447258654,11,True,1447258654_weather_data.json,Rain
4,427,857,9.94,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exit 6 Jamaica Ave),594,1447258654_speed_data.txt,1447258654,11,True,1447258654_weather_data.json,Rain


Now, let's export to CSV.

In [11]:
traffic_and_weather_df.to_csv("traffic_and_weather_df.csv")